# Kaggle MNIST
## A guide to DNN with Tensorflow in the Google Cloud

## Accounts Setup
### Kaggle
Setup an account at [kaggle](https://www.kaggle.com/?login=true), **requires valid mobile number**
### Google Cloud
https://console.cloud.google.com/freetrial, **requires credit card**

## Challenge
Take a look at the [Digit Recognizer](https://www.kaggle.com/c/digit-recognizer) challenge.
![10k images](https://www.tensorflow.org/images/mnist_10k_sprite.png)
from [https://www.tensorflow.org/images/mnist_10k_sprite.png](https://www.tensorflow.org/images/mnist_10k_sprite.png) 

## Google Cloud Setup

Create a new Project via [projects](https://console.cloud.google.com/project) and enable billing with [billing support](https://support.google.com/cloud/answer/6293499#enable-billing).

(Optional: enable ML Apis for the project [enableapi](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,dataflow,compute_component,logging,storage_component,storage_api,bigquery)).

Setup Compute Engine [https://cloud.google.com/compute/docs/gcloud-compute/](https://cloud.google.com/compute/docs/gcloud-compute/)

Create Project, enable billing
https://cloud.google.com/datalab/docs/quickstarts

Open your cloud shell
![https://cloud.google.com/shell/docs/images/shellstart-lowres.gif](https://cloud.google.com/shell/docs/images/shellstart-lowres.gif)
([https://cloud.google.com/shell/](https://cloud.google.com/shell/))

```bash
# scripted:
gcloud init

# or manual:
gcloud projects list
gcloud config set core/project <project-id>
gcloud config set compute/zone europe-west1-b
```

## Datalab Setup

In the cloud shell:
```bash
datalab create <lab-id>
>>> -> [10] europe-west1-b
>>>   Generating public/private rsa key pair.
>>>   Enter passphrase (empty for no passphrase):
>>> -> web preview -> select port 8081
```

The Datalab opens in a new tab.

Create a folder notebooks.

## Tensorflow Setup

Tensorflow is already included in Datalab.

## Datalab Simple ANN Notebook

Download from current repository:
[Simple ANN](./kaggle-mnist-ann.ipynb)

and upload it to the Datalab notebooks folder.

Open the notebook.

- edit mode:
    - **ctrl + enter**: evaluate cell
    - **shift + enter**: evaluate cell and go to next cell
- control mode:
    - **up, down**: select cell
    - **h**: all the info you need
    - **p**: menu

## Tutorial TensorFlow and Deep Learning without a PhD

Verify the notebook with [TensorFlow and Deep Learning without a PhD](https://codelabs.developers.google.com/codelabs/cloud-tensorflow-mnist) step 6.

Continue with step 7, 8, ...

## Datalab CNN Notebook

Download from current repository:
[Simple ANN](./kaggle-mnist-cnn.ipynb)

and upload it to the Datalab notebooks folder.

Open the notebook.

This notebook is based on the [Deep MNIST for Experts](https://www.tensorflow.org/get_started/mnist/pros).

## Google Cloud VM Jupyter Notebook with GPU Setup

### Intro
Google Cloud provides VMs with Tesla K80 GPUs, see [GPUs are now available for Google Compute Engine and Cloud Machine Learning](https://cloudplatform.googleblog.com/2017/02/GPUs-are-now-available-for-Google-Compute-Engine-and-Cloud-Machine-Learning.html). 

GPU are supported in the regions and zones [europe-west1-b and europe-west1-d](https://cloud.google.com/compute/docs/gpus/).

### Request GPU

Go to [quotas](https://console.cloud.google.com/compute/quotas), upgrade and apply for 1 gpu in europe-west1.

### VM and Cuda
For follow the next lines and consult [Running Jupyter notebooks on GPU on Google Cloud](https://medium.com/google-cloud/running-jupyter-notebooks-on-gpu-on-google-cloud-d44f57d22dbd) for the details:

Open your cloud shell
```bash
gcloud components list

# if beta is not installed:
gcloud components update && gcloud components install beta

# did not work
gcloud beta compute regions describe europe-west1-b

gcloud beta compute instances create kaggle-mnist-gpu-learner --machine-type n1-standard-2 --zone europe-west1-b --accelerator type=nvidia-tesla-k80,count=1 --image-family ubuntu-1604-lts --image-project ubuntu-os-cloud --boot-disk-size 200GB --maintenance-policy TERMINATE --restart-on-failure

-> WARNING: You have selected a disk size of under [200GB]. This may result in poor I/O performance. For more information, see: https://developers.google.com/compute/docs/disks#pdperformance.
ERROR: (gcloud.beta.compute.instances.create) Could not fetch resource:
 - Quota 'NVIDIA_K80_GPUS' exceeded. Limit: 0.0
-> https://console.cloud.google.com/compute/quotas

gcloud compute ssh kaggle-mnist-gpu-learner --zone europe-west1-b

    ## on the vm-instance kaggle-mnist-gpu-learner
    curl -O http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_8.0.61-1_amd64.deb
    sudo dpkg -i cuda-repo-ubuntu1604_8.0.61-1_amd64.deb
    sudo apt-get update
    rm cuda-repo-ubuntu1604_8.0.61-1_amd64.deb
    # next command will take some minutes
    sudo apt-get install cuda -y
    echo 'export CUDA_HOME=/usr/local/cuda' >> ~/.bashrc
    echo 'export PATH=$PATH:$CUDA_HOME/bin' >> ~/.bashrc
    echo 'export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CUDA_HOME/lib64' >> ~/.bashrc
    source ~/.bashrc
```
download cuDNN 8.0/5.1 for linux and upload to instance
```bash
    ## on the vm-instance kaggle-mnist-gpu-learner
    tar xzvf cudnn-8.0-linux-x64-v5.1.tgz
    sudo cp cuda/lib64/* /usr/local/cuda/lib64/
    sudo cp cuda/include/cudnn.h /usr/local/cuda/include/
    rm -rf ~/cuda
    rm cudnn-8.0-linux-x64-v5.1.tgz
    
    # test
    nvidia-smi
    
    curl -O https://repo.continuum.io/archive/Anaconda3-4.3.1-Linux-x86_64.sh
    bash Anaconda3-4.3.1-Linux-x86_64.sh
        
        Do you wish the installer to prepend the Anaconda3 install location
        to PATH in your /home/your_username/.bashrc ? [yes|no]
        [no] >>> yes
        
    rm Anaconda3-4.3.1-Linux-x86_64.sh
    
    sudo reboot
```

### Tensorflow
```
gcloud compute ssh kaggle-mnist-gpu-learner --zone europe-west1-b

    ## on the vm-instance kaggle-mnist-gpu-learner
    pip install tensorflow-gpu
    pip install keras

```

### Jupyter Notebook
```
    jupyter notebook --generate-config
    vi /home/your_username/.jupyter/jupyter_notebook_config.py
```
prepend contents from https://gist.githubusercontent.com/durgeshm/3d2c7317c61437cb8d015f45b7b8ea36/raw/dfc070e0e9064deca7a59631b6a3808c2a3ddaf6/jupyter_notebook_config.py
and prepend
```
## The IP address the notebook server will listen on.
c.NotebookApp.ip = '0.0.0.0'
```

continue by creating ssl keys and cert (TODO: move cert/key to jupyter_notebook_config.py)

```bash
    ## on the vm-instance kaggle-mnist-gpu-learner
    # notebook http://jupyter-notebook.readthedocs.io/en/latest/public_server.html
    openssl req -x509 -nodes -days 365 -newkey rsa:1024 -keyout mykey.key -out mycert.pem
    jupyter notebook --certfile=mycert.pem --keyfile mykey.key kaggle-mnist/
    
```

### Firewall Rule 8888
create firewall rule https://console.cloud.google.com/networking/firewalls/list?project=project_id&tab=INGRESS

rule config: Target tags: jupyter, IP ranges: 0.0.0.0/0, Source filters: tcp:8888, Action on match: Allow, Direction: Ingress

### Connect to Notebook
get ip from https://console.cloud.google.com/compute/instances?project=kaggle-mnist-gpu

and connect to https://vm_ext_id:8888/

have fun!

# The End

## Stopping instances

from cloud shell:
```bash
datalab stop <lab-id>

gcloud compute instances stop kaggle-mnist-gpu-learner
```

## Starting instances
from cloud shell:
```bash
datalab list
datalab connect <lab-id>

gcloud compute instances start kaggle-mnist-gpu-learner
```

# Links and References

## MNIST

- [MNIST Home](http://yann.lecun.com/exdb/mnist/)

## Notebooks

- [TensorFlow deep NN](https://www.kaggle.com/kakauandme/tensorflow-deep-nn)
- [CNN on MNIST with TensorFlow (~99.1)](https://www.kaggle.com/jianchengyang/cnn-on-mnist-with-tensorflow)

## Demos

- [ConvNetJS MNIST demo](https://cs.stanford.edu/people/karpathy/convnetjs/demo/mnist.html)
- [Tensorflow Playground](http://playground.tensorflow.org)

## Tutorials

- [Not another MNIST tutorial with TensorFlow](https://www.oreilly.com/learning/not-another-mnist-tutorial-with-tensorflow)
- [TensorFlow and Deep Learning without a PhD](https://codelabs.developers.google.com/codelabs/cloud-tensorflow-mnist)